# Face mask detection 

In [1]:
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import torch
import torchvision
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from PIL import Image, ImageDraw
import xml.etree.ElementTree as ET
import cv2

images_dir = './data/images/'
annotations_dir = './data/annotations/'

In [ ]:
print(len())